## Preliminaries

In [1]:
# Import necessary libraries
import geopandas as gpd
import pandas as pd
import os

In [2]:
os.chdir("C:/Users/emshe/Desktop/BRAINSTATION/CAPSTONE/GIT_REPO")

## Helper functions

In [3]:
# Define function to examine dataframes

def examine_df(name,df):
    """
    Check basic info about a dataframe df
    """
    
    print(f"\n\nNumber of records in the {name} is: {len(df)}\n")
    print(f"The columns in the {name} are: {df.columns}\n")
    print(f"\n Other info about {name}:")
    display(df.info())
    print(f"\n\nSample of records in the {name}:")
    display(df.head(5))

## Load and examine geographic files

In [4]:
# Load the shapefiles
zones_gdf = gpd.read_file("DATA/GEOGRAPHIC/ZONE_2023/Zone_2023/Zone_2023.shp")
nbhds_gdf = gpd.read_file("DATA/GEOGRAPHIC/NBHD_2023/Nbhd_2023/Nbhd_2023.shp")
tracts_gdf = gpd.read_file("DATA/GEOGRAPHIC/TRACT_2021/tract_2021.shp")

In [5]:
# Import economic data csvs
rent_by_tract = pd.read_csv("C:/Users/emshe/Desktop/BRAINSTATION/CAPSTONE/GIT_REPO/DATA/ECONOMIC/PROCESSED_DATAFRAMES/avg_rent_by_tract.csv", dtype={'Tract': str})
rent_by_nbhd = pd.read_csv("C:/Users/emshe/Desktop/BRAINSTATION/CAPSTONE/GIT_REPO/DATA/ECONOMIC/PROCESSED_DATAFRAMES/avg_rent_by_neigh.csv")


In [11]:
# Examine shapefiles

gdfs = {"Zones Dataframe":zones_gdf,"Neighborhood dataframe":nbhds_gdf, "Census tract dataframe": tracts_gdf}
for name,df in gdfs.items():
    examine_df(name,df)



Number of records in the Zones Dataframe is: 521

The columns in the Zones Dataframe are: Index(['zone', 'zone_geometry'], dtype='object')


 Other info about Zones Dataframe:
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 521 entries, 0 to 520
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   zone           521 non-null    object  
 1   zone_geometry  521 non-null    geometry
dtypes: geometry(1), object(1)
memory usage: 8.3+ KB


None



Sample of records in the Zones Dataframe:


,zone,zone_geometry
0,Abbotsford,"POLYGON ((-13618744.996 6293386.567, -13618650..."
1,Mission,"POLYGON ((-13621492.351 6336339.105, -13621109..."
2,Barrie,"POLYGON ((-8881653.78 5564338.457, -8881238.81..."
3,City of Belleville,"POLYGON ((-8586806.986 5526375.747, -8585636.5..."
4,City of Quinte West,"POLYGON ((-8633033.476 5539563.948, -8632904.4..."




Number of records in the Neighborhood dataframe is: 986

The columns in the Neighborhood dataframe are: Index(['nbhd', 'nbhd_geometry'], dtype='object')


 Other info about Neighborhood dataframe:
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 986 entries, 0 to 985
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   nbhd           986 non-null    object  
 1   nbhd_geometry  986 non-null    geometry
dtypes: geometry(1), object(1)
memory usage: 15.5+ KB


...

,nbhd,nbhd_geometry
0,Mill Lake,"POLYGON ((-13614863.669 6283280.399, -13614864..."
1,Abbotsford/McMillan,"POLYGON ((-13613246.325 6283847.766, -13613059..."
2,Townline/Clearbrook,"POLYGON ((-13613010.778 6288767.654, -13612878..."
3,Mt. Lehman/Aberdeen/Poplar,"POLYGON ((-13621010.048 6293378.663, -13621041..."
4,Whatcom/Sumas,"POLYGON ((-13604657.482 6297077.561, -13604631..."




Number of records in the Census tract dataframe is: 6247

The columns in the Census tract dataframe are: Index(['tract', 'tract_geometry'], dtype='object')


 Other info about Census tract dataframe:
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 6247 entries, 0 to 6246
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   tract           6247 non-null   object  
 1   tract_geometry  6247 non-null   geometry
dtypes: geometry(1), object(1)
memory usage: 97.7+ KB


None



Sample of records in the Census tract dataframe:


,tract,tract_geometry
0,5370001.08,"POLYGON ((7196507.366 869787.991, 7196501.617 ..."
1,0010002.00,"POLYGON ((8980216.643 2151065.36, 8980377.609 ..."
2,5370001.09,"POLYGON ((7196437.003 869160.246, 7196434.403 ..."
3,5370120.02,"POLYGON ((7189475.703 865662.849, 7189448.943 ..."
4,0010006.00,"POLYGON ((8980091.143 2152478.609, 8980100.254..."


## Clean and Preprocess GDFs

In [10]:
# Drop unnecessary columns from gdfs and rename remaining columns

# Drop columns from zone gdf
zone_columns = ['ZONENAME_E', 'geometry']
zones_gdf = zones_gdf[zone_columns]

# Rename columns
zones_gdf.rename(columns = {'ZONENAME_E': 'zone', 'geometry':'zone_geometry'},inplace = True)

# Drop columns from neighborhood gdf
nbhd_columns = ['NBHDNAME_E', 'geometry']
nbhds_gdf = nbhds_gdf[nbhd_columns]

# Rename columns
nbhds_gdf.rename(columns = {'NBHDNAME_E': 'nbhd', 'geometry':'nbhd_geometry'},inplace = True)

# Drop columns from neighborhood gdf
tract_columns = ['CTUID', 'geometry']
tracts_gdf = tracts_gdf[tract_columns]

# Rename columns
tracts_gdf.rename(columns = {'CTUID': 'tract', 'geometry':'tract_geometry'},inplace = True)


## Restrict neighborhoods and census tracts to match CMHC economic data

In [50]:
# Define the corrections in a dictionary
neighborhood_renames = {
    'Hastings/Grandview/Woodlands': 'Hastings/Sunrise/Grandview/Woodlands',
    'Dundrave/W Vancouver Remainder': 'Dundrave/West Vancouver Remainder'
}

# Apply the replacements
van_nbhds_df['NBHDNAME_E'] = van_nbhds_df['NBHDNAME_E'].replace(neighborhood_renames)


In [76]:
# Restrict to neighborhoods appearing in CMHC economic data
CMHC_van_nbhds_df = van_nbhds_df[van_nbhds_df['NBHDNAME_E'].isin(avg_rent_by_nbhd['Neigh'])]
examine_df('CMHC Neighborhood DF',CMHC_van_nbhds_df)



Number of records in the CMHC Neighborhood DF is: 68

The columns in the CMHC Neighborhood DF are: Index(['METNBHD_UI', 'METCODE', 'NBHDCODE', 'NBHDNAME_E', 'ZONECODE',
       'nbhd_geometry'],
      dtype='object')


 Other info about CMHC Neighborhood DF:
<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 68 entries, 753 to 822
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   METNBHD_UI     68 non-null     object  
 1   METCODE        68 non-null     object  
 2   NBHDCODE       68 non-null     object  
 3   NBHDNAME_E     68 non-null     object  
 4   ZONECODE       68 non-null     object  
 5   nbhd_geometry  68 non-null     geometry
dtypes: geometry(1), object(5)
memory usage: 3.7+ KB


None



Sample of records in the CMHC Neighborhood DF:


,METNBHD_UI,METCODE,NBHDCODE,NBHDNAME_E,ZONECODE,nbhd_geometry
753,2410010,2410,010,West End/Stanley Park North,01,"POLYGON ((-13707903.991 6324277.893, -13707728..."
754,2410020,2410,020,West End/Stanley Park South,01,"POLYGON ((-13709954.808 6328768.906, -13708979..."
755,2410030,2410,030,English Bay,02,"POLYGON ((-13708084.955 6323746.439, -13707920..."
756,2410040,2410,040,Downtown Central,03,"POLYGON ((-13706769.27 6325491.913, -13706673...."
757,2410050,2410,050,North False Creek,03,"POLYGON ((-13705341.854 6321743.569, -13705338..."


## Use geometry columns to merge shapefile datasets

In [27]:
# Perform spatial join: each neighborhood gets its parent zone's info

# Ensure CRS matches
van_nbhds_df = van_nbhds_df.to_crs(van_zones_df.crs)


van_nbhds_with_zone = gpd.sjoin(
    van_nbhds_df,
    van_zones_df[['ZONECODE', 'ZONENAME_E', 'zone_geometry']],
    how='left',
    predicate='within'
)